# Advent of Code 2022

https://adventofcode.com/2022

In [1]:
# A few standard library imports
import itertools, math, collections, os, functools, json, time, re, bisect
# nice printing
from IPython.display import display, clear_output, HTML

In [2]:
def getData(day):
    if os.path.exists("data-2022/day%dinput.txt" %day):
        with open("data-2022/day%dinput.txt" %day, 'r') as f:
            return f.read()
    import browsercookie
    import urllib.request
    cj = browsercookie.firefox()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    assert b'watercrossing' in opener.open("https://adventofcode.com/2022/").read()
    data = opener.open("https://adventofcode.com/2022/day/%d/input" %day).read().decode('utf-8')
    with open("data-2022/day%dinput.txt" %day, 'w') as f:
        f.write(data)
    return data

## Day 1

In [3]:
day1data = [[int(y) for y in x.split()] for x in getData(1).split("\n\n")]

In [4]:
max(sum(x) for x in day1data)

64929

In [5]:
sum(sorted(sum(x) for x in day1data)[-3:])

193697

## Day 2

In [6]:
day2data = [(ord(x.split(" ")[0]) - 64, ord(x.split(" ")[1]) - 87) for x in getData(2).split("\n") if x]

In [7]:
sum(x[1] for x in day2data) + sum(any((x[1] - x[0]) == y for y in (1, -2))*6 + (x[1] == x[0])*3 for x in day2data)

12645

In [8]:
def day2Choice(oth, obj):
    if obj == 2: return oth
    if obj == 1: return oth - 1 if oth > 1 else 3
    if obj == 3: return oth + 1 if oth < 3 else 1

In [9]:
sum(day2Choice(*x) for x in day2data) + sum((x[1] - 1)*3 for x in day2data)

11756

## Day 3

In [10]:
day3data = [[ord(y) - 96 if ord(y) > 96 else ord(y) - 38 for y in x] for x in getData(3).split("\n") if x]

In [11]:
sum(sum(set(x[:len(x)//2]) & set(x[len(x)//2:])) for x in day3data)

8053

In [12]:
sum(sum(set(day3data[i]) & set(day3data[i+1]) & set(day3data[i+2])) for i in range(0, len(day3data), 3))

2425

## Day 4

In [13]:
day4data = [[range(int(y.split("-")[0]), int(y.split("-")[1]) + 1) for y in x.split(",")] for x in getData(4).split("\n") if x]

In [14]:
sum(len(set(x[0]).union(set(x[1]))) == max(len(x[0]), len(x[1])) for x in day4data)

651

In [15]:
sum(len(set(x[0]).intersection(set(x[1]))) > 0 for x in day4data)

956

## Day 5

In [16]:
day5stacks =  getData(5).split("\n\n")[0].split("\n")[:-1]
day5stacks = [[x[i] for x in day5stacks if x[i] != ' '] for i in range(1, len(day5stacks[0]), 4)]

In [17]:
day5instructions = [tuple(int(x.split(" ")[i]) for i in [1, 3, 5]) for x in getData(5).split("\n\n")[1].split("\n") if x]

In [18]:
for num, origin, dest in day5instructions:
    for i in range(num):
        day5stacks[dest - 1].insert(0, day5stacks[origin - 1].pop(0))
"".join(x[0] for x in day5stacks)

'SVFDLGLWV'

In [19]:
day5stacks =  getData(5).split("\n\n")[0].split("\n")[:-1]
day5stacks = [[x[i] for x in day5stacks if x[i] != ' '] for i in range(1, len(day5stacks[0]), 4)]

In [20]:
for num, origin, dest in day5instructions:
    for i in range(num):
        day5stacks[dest - 1].insert(i, day5stacks[origin - 1].pop(0))
"".join(x[0] for x in day5stacks)

'DCVTCVPCL'

## Day 6

In [21]:
day6data = getData(6).strip()

In [22]:
next(i for i in range(4, len(day6data)) if len(set(day6data[i-4:i])) == 4)

1282

In [23]:
next(i for i in range(14, len(day6data)) if len(set(day6data[i-14:i])) == 14)

3513

## Day 7

In [24]:
day7data = getData(7).strip().split("\n")

In [25]:
class Folder(object):
    def __init__(self, name, parent):
        self.name = name
        self.files = {} # name: size
        self.folders = {} # name: Folder objects
        self.parent = parent
    def prettyString(self):
        return"\n".join(self._format())
    def _format(self):
        s = ["- %s (dir)" %self.name]
        s.extend("  " + y for x in self.folders.values() for y in x._format())
        s.extend("  - %s (file, size=%d)" %(fn, size) for fn, size in self.files.items())
        return s
    def folderSize(self):
        return sum(self.files.values()) + sum(x.folderSize() for x in self.folders.values())
    def getChildFolders(self):
        return itertools.chain([self], *(x.getChildFolders() for x in self.folders.values()))
        
def parseStream(day7data):
    root = cwd = Folder('/', None)
    i = 1
    day7datalen = len(day7data)
    while i < day7datalen:
        line = day7data[i]
        if line == '$ ls':
            while i+1 < day7datalen and not (line := day7data[i+1]).startswith("$ "):
                if line.startswith("dir "):
                    cwd.folders[line.split("dir ", 1)[1]] = Folder(line.split("dir ", 1)[1], cwd)
                else:
                    cwd.files[line.split(" ", 1)[1]] = int(line.split(" ", 1)[0])
                i += 1
        elif line == '$ cd ..':
            cwd = cwd.parent
        elif line == '$ cd /':
            cwd = root
        elif line.startswith("$ cd "):
            cwd = cwd.folders[line.split("$ cd ", 1)[1]]
        else:
            print("ERROR: %s" %line)
        i += 1
    return root

In [26]:
root = parseStream(day7data)

In [27]:
print(root.prettyString())

- / (dir)
  - gts (dir)
    - grwwbrgz.wft (file, size=846)
    - mrnhn.psz (file, size=72000)
    - qvnbd.dqs (file, size=155241)
    - tndtmwfv (file, size=6655)
  - lwhbw (dir)
    - lrrl.lth (file, size=99946)
  - pcqjnl (dir)
    - gljcvm (dir)
      - tmwzlzn (file, size=264381)
    - lqwntmdg (dir)
      - jjfwr (dir)
        - cfhjvmh (dir)
          - gzfgc (dir)
            - cfhjvmh.wwh (file, size=134989)
      - rfqbmb (dir)
        - cbrvhz (dir)
          - wdtm.rjr (file, size=131072)
        - flcw (dir)
          - wlfwpb.wpg (file, size=216675)
        - mnd (dir)
          - hzzzzvmr.lsz (file, size=28976)
    - lrrl (dir)
      - cpmvnf (dir)
        - srtqvcv (dir)
          - mrnhn (dir)
            - fbrwd (dir)
              - nqth.gcn (file, size=163166)
      - dcfmtw (dir)
        - nzpdtfr (dir)
          - qwtwps (dir)
            - cmf (dir)
              - wdsjg.thm (file, size=73595)
          - vcthd (dir)
            - cfhjvmh (file, size=15016)
     

In [28]:
sum(s for f in root.getChildFolders() if (s := f.folderSize()) <= 100000)

1778099

In [29]:
desiredSize = 30000000 - (70000000 - root.folderSize())
sorted(s for f in root.getChildFolders() if (s := f.folderSize()) >= desiredSize)[0]

1623571

## Day 8

In [30]:
day8data = getData(8).strip().split("\n")

In [31]:
visibleTrees = list(itertools.product((0, len(day8data)), range(len(day8data[0])))) + \
                list(itertools.product(range(1, len(day8data) - 1), (0, len(day8data[0]))))
DIR = ((0, 1), (1, 0), (0, -1), (-1, 0))
for i, j in itertools.product(range(1, len(day8data) - 1), range(1, len(day8data[0]) - 1)):
    for diri, dirj in DIR:
        iloc, jloc = i + diri, j + dirj
        isVisible = True
        while isVisible and iloc not in (-1, len(day8data)) and jloc not in (-1, len(day8data[0])):
            isVisible = int(day8data[i][j]) > int(day8data[iloc][jloc])
            iloc, jloc = iloc + diri, jloc + dirj
        if isVisible:
            visibleTrees.append((i, j))
            break 
len(visibleTrees) 

1533

In [32]:
scenicScores = []
for i, j in itertools.product(range(1, len(day8data) - 1), range(1, len(day8data[0]) - 1)):
    thisScore = []
    for diri, dirj in ((0, 1), (1, 0), (0, -1), (-1, 0)):
        iloc, jloc = i + diri, j + dirj
        while iloc not in (-1, len(day8data)) and jloc not in (-1, len(day8data[0])) and \
                int(day8data[i][j]) > int(day8data[iloc][jloc]):
            iloc, jloc = iloc + diri, jloc + dirj
        thisScore.append(abs(min(max(iloc, 0), len(day8data) -1) - i) + abs(min(max(jloc, 0), len(day8data[0]) -1) - j))
    scenicScores.append(functools.reduce(lambda x, y: x*y, thisScore, 1))
max(scenicScores)

345744

## Day 9

In [33]:
day9data = [(x.split(" ")[0], int(x.split(" ")[1])) for x in getData(9).strip().split("\n")]

In [34]:
tailPositions = [[0, 0]]
headPosition = [0, 0]
for direction, steps in day9data:
    for i in range(steps):
        newTail = [x for x in tailPositions[-1]]
        if direction in ('U', 'D'):
            headPosition[1] += 1 if direction == 'U' else -1
            if abs(yOff := headPosition[1] - newTail[1]) > 1:
                newTail[1] += int(yOff/abs(yOff))
                if (xOff := headPosition[0] - newTail[0]) != 0:
                    newTail[0] += int(xOff/abs(xOff))
        elif direction in ('R', 'L'):
            headPosition[0] += 1 if direction == 'R' else -1
            if abs(xOff := headPosition[0] - newTail[0]) > 1:
                newTail[0] += int(xOff/abs(xOff))
                if (yOff := headPosition[1] - newTail[1]) != 0:
                    newTail[1] += int(yOff/abs(yOff))
        tailPositions.append(newTail)

In [35]:
len(set(tuple(x) for x in tailPositions))

6087

In [36]:
knotPositions = [[0, 0] for x in range(10)]
tailPositions = [[0, 0]]
for direction, steps in day9data:
    for i in range(steps):
        if direction in ('U', 'D'):
            knotPositions[0][1] += 1 if direction == 'U' else -1
        elif direction in ('R', 'L'):
            knotPositions[0][0] += 1 if direction == 'R' else -1           
        for i in range(0, 9):
            if abs(yOff := knotPositions[i][1] - knotPositions[i + 1][1]) > 1:
                knotPositions[i + 1][1] += int(yOff/abs(yOff))
                if (xOff := knotPositions[i][0] - knotPositions[i + 1][0]) != 0:
                    knotPositions[i + 1][0] += int(xOff/abs(xOff))
            if abs(xOff := knotPositions[i][0] - knotPositions[i + 1][0]) > 1:
                knotPositions[i + 1][0] += int(xOff/abs(xOff))
                if (yOff := knotPositions[i][1] - knotPositions[i + 1][1]) != 0:
                    knotPositions[i + 1][1] += int(yOff/abs(yOff))
        tailPositions.append([x for x in knotPositions[-1]])
        

In [37]:
len(set(tuple(x) for x in tailPositions))

2493

In [38]:
[x([z[i] for z in tailPositions]) for x, i in itertools.product([min, max], [0, 1])]

[-204, -7, 65, 207]

In [39]:
## hmm, a 269 x 214 grid is too large to visualise really :(

## Day 10

In [40]:
day10data = [(x.split(" ")[0], int(x.split(" ")[1]) if len(x.split(" ")) > 1 else None) 
             for x in getData(10).strip().split("\n")]

In [41]:
state = [1]
for op, num in day10data:
    if op == 'noop':
        state.append(state[-1])
    elif op == 'addx':
        state.extend([state[-1], state[-1] + num])

In [42]:
sum([i*state[i-1] for i in range(20,len(state), 40)])

14920

In [43]:
print("\n".join("".join(["#" if abs(s - (i % 40)) < 2 else " " 
                         for i, s in enumerate(state)])[(j*40):(j+1)*40] for j in range(len(state)//40)))

###  #  #  ##   ##   ##  ###  #  # #### 
#  # #  # #  # #  # #  # #  # #  #    # 
###  #  # #    #  # #    ###  #  #   #  
#  # #  # #    #### #    #  # #  #  #   
#  # #  # #  # #  # #  # #  # #  # #    
###   ##   ##  #  #  ##  ###   ##  #### 


## Day 11

In [44]:
def parseOp(op):
    if op == 'old * old': 
        return lambda x: x*x
    elif len(z := op.split('old * ')) > 1:
        return lambda x: x * int(z[1])
    elif len(z := op.split('old + ')) > 1:
        return lambda x: x + int(z[1])
    else:
        raise Exception("unknown op %s" %op)

In [45]:
day11Data = [{'op' : parseOp((y := x.split("\n"))[2].split(" = ")[1]),
              'initialItems': [int(z) for z in y[1].split(": ")[1].split(", ")],  
              'test' : int(y[3].split(" by ")[1]),
              'true': int(y[4].split('to monkey ')[1]),
              'false': int(y[5].split('to monkey ')[1])} 
             for x in getData(11).split("\n\n")]

In [46]:
for m in day11Data:
    m['items'] = [x for x in m['initialItems']]
    m['count'] = 0
for r in range(20):
    for m in day11Data:
        while m['items']:
            m['count'] += 1
            i = m['op'](m['items'].pop(0)) // 3
            day11Data[m['true'] if i % m['test'] == 0 else m['false']]['items'].append(i)

In [47]:
[a*b for a, b in [sorted([x['count'] for x in day11Data])[-2:]]][0]

54752

In [48]:
lcm = functools.reduce(lambda x, y: x*y, [x['test'] for x in day11Data], 1)

In [49]:
for m in day11Data:
    m['items'] = [x for x in m['initialItems']]
    m['count'] = 0
for r in range(10000):
    for m in day11Data:
        while m['items']:
            m['count'] += 1
            i = m['op'](m['items'].pop(0)) % lcm
            day11Data[m['true'] if i % m['test'] == 0 else m['false']]['items'].append(i)

In [50]:
[a*b for a, b in [sorted([x['count'] for x in day11Data])[-2:]]][0]

13606755504

## Day 12

In [51]:
day12Data = getData(12).strip().split("\n")

In [52]:
startX, startY = [(xi, yi) for xi in range(len(day12Data)) for yi in range(len(day12Data[0])) if day12Data[xi][yi] == 'S'][0]
endX, endY = [(xi, yi) for xi in range(len(day12Data)) for yi in range(len(day12Data[0])) if day12Data[xi][yi] == 'E'][0]

In [53]:
distanceMap = [[0 if day12Data[x][y] == 'S' else -1 for y in range(len(day12Data[0]))] for x in range(len(day12Data))]
toprocess = [(startX, startY)]
while toprocess:
    xi, yi = toprocess.pop(0)
    for dx, dy in ((0, 1), (1, 0), (0, -1), (-1, 0)):
        if (xi + dx in range(len(day12Data)) and yi + dy in range(len(day12Data[0])) 
            and ((day12Data[dx+xi][dy+yi] in ('a', 'b') and day12Data[xi][yi]  == 'S') or
                 (day12Data[dx+xi][dy+yi] == 'E' and day12Data[xi][yi]) in ('z', 'y') or 
                 (day12Data[dx+xi][dy+yi] != 'E' and ord(day12Data[dx+xi][dy+yi]) - ord(day12Data[xi][yi]) < 2))):
            if distanceMap[dx+xi][dy+yi] == -1 or distanceMap[dx+xi][dy+yi] > distanceMap[xi][yi] + 1:
                distanceMap[dx+xi][dy+yi] = distanceMap[xi][yi] + 1
                toprocess.append((dx+xi, dy+yi))

In [54]:
distanceMap[endX][endY]

447

In [55]:
distanceMap = [[0 if day12Data[x][y] in ('S', 'a') else -1 for y in range(len(day12Data[0]))] for x in range(len(day12Data))]
toprocess = [(x, y) for x in range(len(day12Data)) for y in range(len(day12Data[0])) if day12Data[x][y] in ('S', 'a')]
while toprocess:
    xi, yi = toprocess.pop(0)
    for dx, dy in ((0, 1), (1, 0), (0, -1), (-1, 0)):
        if (xi + dx in range(len(day12Data)) and yi + dy in range(len(day12Data[0])) 
            and ((day12Data[dx+xi][dy+yi] in ('a', 'b') and day12Data[xi][yi]  == 'S') or
                 (day12Data[dx+xi][dy+yi] == 'E' and day12Data[xi][yi]) in ('z', 'y') or 
                 (day12Data[dx+xi][dy+yi] != 'E' and ord(day12Data[dx+xi][dy+yi]) - ord(day12Data[xi][yi]) < 2))):
            if distanceMap[dx+xi][dy+yi] == -1 or distanceMap[dx+xi][dy+yi] > distanceMap[xi][yi] + 1:
                distanceMap[dx+xi][dy+yi] = distanceMap[xi][yi] + 1
                toprocess.append((dx+xi, dy+yi))

In [56]:
distanceMap[endX][endY]

446

## Day 13 

In [57]:
day13Data = [[json.loads(y) for y in x.split("\n")] for x in getData(13).strip().split("\n\n")]

In [58]:
def compare(x, y):
    """-1 left lower, 0 equal, 1 right lower"""
    if isinstance(x, int):
        if isinstance(y, int):
            if x < y: return -1
            if x == y: return 0
            return 1
        else:
            return compare([x], y)
    else:
        if isinstance(y, int):
            return compare(x, [y])
        else:
            if not x and not y: return 0
            if not x and y: return -1
            if not y and x: return 1
            for i in range(max(len(x), len(y))):
                ret = compare(x[i], y[i])
                if ret != 0: return ret
                if i+1 == len(x):
                    if i+1 == len(y): return 0
                    return -1
                if i+1 == len(y): return 1

In [59]:
sum(i+1 for i,x in enumerate(day13Data) if compare(x[0], x[1]) == -1)

5503

In [60]:
day13Sorted = sorted([x for y in day13Data for x in y] + [[[2]], [[6]]], key=functools.cmp_to_key(compare))

In [61]:
(day13Sorted.index([[2]]) + 1) * (day13Sorted.index([[6]]) + 1)

20952

## Day 14

In [62]:
day14Data = [[[int(z) for z in y.split(",")] for y in x.split(" -> ")] for x in getData(14).strip().split("\n")]

In [63]:
xmin, xmax = min(y[0] for x in day14Data for y in x), max(y[0] for x in day14Data for y in x)
ymin, ymax = min(y[1] for x in day14Data for y in x), max(y[1] for x in day14Data for y in x)
xmin, xmax, ymin, ymax

(496, 584, 16, 157)

In [64]:
cave = [[' ' for yi in range(xmax-xmin+1)] for xi in range(ymax+1)]
for wall in day14Data:
    for (wallSX, wallSY), (wallEX, wallEY) in zip(wall, wall[1:]):
        xdir = (wallEX-wallSX)//abs(wallEX-wallSX) if wallEX-wallSX else 1
        ydir = (wallEY-wallSY)//abs(wallEY-wallSY) if wallEY-wallSY else 1
        for xi, yi in itertools.product(range(wallSX-xmin, wallEX-xmin + xdir, xdir), range(wallSY, wallEY + ydir, ydir)):
            cave[yi][xi] = '#'
cave[0][500-xmin] = '+'

In [65]:
print("\n".join(["".join(y) for y in cave]))

    +                                                                                    
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
          

In [66]:
i = 0
try:
    while True:
        s = [0, 500 - xmin] # (y, x) yay
        while True:
            if cave[s[0] + 1][s[1]] == ' ':
                s[0] += 1
            elif cave[s[0] + 1][s[1] - 1] == ' ':
                s = [s[0] + 1, s[1] - 1]
            elif cave[s[0] + 1][s[1] + 1] == ' ':
                s = [s[0] + 1, s[1] + 1]
            else:
                cave[s[0]][s[1]] = 'o'
                i += 1
                break
except IndexError:
    pass

In [67]:
print(i)

1513


In [68]:
print("\n".join(["".join(y) for y in cave]))

    +                                                                                    
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
                                                                                         
          

In [69]:
cave = [[' ' for xi in range(ymax*2 + 4 + 1)] for yi in range(ymax+2+1)]
for wall in day14Data:
    for (wallSX, wallSY), (wallEX, wallEY) in zip(wall, wall[1:]):
        xdir = (wallEX-wallSX)//abs(wallEX-wallSX) if wallEX-wallSX else 1
        ydir = (wallEY-wallSY)//abs(wallEY-wallSY) if wallEY-wallSY else 1
        for xi, yi in itertools.product(range(wallSX + ymax - 500 + 2 , wallEX + ymax + xdir - 500 + 2, xdir), 
                                        range(wallSY, wallEY + ydir, ydir)):
            cave[yi][xi] = '#'
for xi in range(ymax*2 + 4 + 1):
    cave[ymax + 2][xi] = '#'
cave[0][(ymax + 1) + 1] = '+'


In [70]:
xmin, xmax,  ymin, ymax

(496, 584, 16, 157)

In [71]:
i = 0
try:
    while True:
        s = [0, (ymax + 1) + 1] # (y, x) yay
        while True:
            if cave[s[0] + 1][s[1]] == ' ':
                s[0] += 1
            elif cave[s[0] + 1][s[1] - 1] == ' ':
                s = [s[0] + 1, s[1] - 1]
            elif cave[s[0] + 1][s[1] + 1] == ' ':
                s = [s[0] + 1, s[1] + 1]
            else:
                i += 1
                if cave[s[0]][s[1]] == '+':
                    raise IndexError
                cave[s[0]][s[1]] = 'o'
                break
except IndexError:
    pass

In [72]:
i

22646

In [73]:
display(HTML("<style>pre { white-space: pre !important; }</style>"))
print("\n".join(["".join(y) for y in cave]))

                                                                                                                                                               +                                                                                                                                                               
                                                                                                                                                              ooo                                                                                                                                                              
                                                                                                                                                             ooooo                                                                                                                                                             
                                        

## Day 15

In [74]:
day15Data = [((int(x.split(", y=")[0].split("x=")[1]), int(x.split(", y=")[1].split(": ")[0])),
(int(x.split(", y=")[1].split("x=")[1]), int(x.split(", y=")[2]))) for x in getData(15).strip().split("\n")]

In [75]:
def simplifyRange(rs):
    outs = []
    for s, e in sorted([(x, y) for x, y in rs if x!= y]):
        if outs and outs[-1][1] + 1 > s:
            outs[-1] = (outs[-1][0], max(e, outs[-1][1]))
        else:
            outs.append((s, e))
    return outs

In [76]:
sum(xmax - xmin for xmin, xmax in simplifyRange(((sX - max(abs(sX-bX) + abs(sY-bY) - abs(sY-2000000), 0)), 
                                                 sX + max(abs(sX-bX) + abs(sY-bY) - abs(sY-2000000), 0)) 
                                                for (sX, sY), (bX, bY) in day15Data))

4883971

In [77]:
def isValid(x, y):
    return all(abs(sX-x) + abs(sY-y) > abs(sX-bX) + abs(sY-bY) for (sX, sY), (bX, bY) in day15Data)

In [78]:
### 17 seconds... that will do
onEdge = next((x, y) for (sX, sY), (bX, bY) in day15Data
              for x in range(min(max(0,sX -(abs(sX-bX) + abs(sY-bY) + 1)), 4000000),
                              min(max(0,sX + abs(sX-bX) + abs(sY-bY) + 1), 4000000))
          for y in [sY + (abs(sX-bX) + abs(sY-bY) + 1 - x + sX), sY - (abs(sX-bX) + abs(sY-bY) + 1 - x + sX)] 
          if y > -1 and y <= 4000000 
          and isValid(x, y))

In [79]:
onEdge, onEdge[1] + 4000000*onEdge[0]

((3172756, 2767556), 12691026767556)

## Day 16

In [80]:
day16Data = dict((x[6:8], (int(x.split("rate=")[1].split(";")[0]), x.split(" ",9)[9].split(", "))) 
                 for x in getData(16).strip().split("\n"))

In [81]:
# build a fully connected graph of distance costs
day16Graph = collections.defaultdict(dict)
for valve, (flow, dests) in day16Data.items():
    for dest in dests:
        day16Graph[valve][dest] = 1
        day16Graph[dest][valve] = 1
added = True
while added:
    added = False
    for start, dests in day16Graph.items():
        for dest, cost in list(dests.items()):
            for otherDest in day16Graph[dest]:
                if otherDest == start:
                    continue
                if otherDest in dests:
                    if dests[otherDest] > cost + day16Graph[dest][otherDest]:
                        added = True
                        dests[otherDest] = cost + day16Graph[dest][otherDest]
                else:
                    added = True
                    dests[otherDest] = cost + day16Graph[dest][otherDest]
# prune useless valves
for valve, (flow, dests) in day16Data.items():
    if flow == 0 and valve != 'AA':
        if valve in day16Graph: del day16Graph[valve]
        for nodes in day16Graph.values(): 
            if valve in nodes: del nodes[valve]

In [82]:
def releasePressure(graph, timeLeft, valvesOpen, currentLocation):
    if timeLeft == 0: return (0, [(timeLeft, currentLocation)])
    options = [releasePressure(graph, timeLeft - cost - 1, valvesOpen + [currentLocation], dest) 
               for dest, cost in graph[currentLocation].items() if dest not in valvesOpen and cost < timeLeft]
    bestOption = max(options, key=lambda x: x[0]) if options else (0, [(timeLeft, "remain here")])
    return (day16Data[currentLocation][0]*timeLeft + bestOption[0], [(timeLeft, currentLocation)] + bestOption[1]) 

In [83]:
releasePressure(day16Graph, 30, [], 'AA')

(1923,
 [(30, 'AA'),
  (27, 'NV'),
  (24, 'PS'),
  (21, 'FX'),
  (18, 'JM'),
  (15, 'KZ'),
  (12, 'UX'),
  (9, 'DO'),
  (6, 'PH'),
  (3, 'DG'),
  (0, 'RG')])

In [84]:
score, steps = releasePressure(day16Graph, 26, [], 'AA')
score2, steps2 = releasePressure(day16Graph, 26, [x[1] for x in steps], 'AA')
score + score2, list(itertools.zip_longest(steps, steps2))

(2594,
 [((26, 'AA'), (26, 'AA')),
  ((23, 'NV'), (23, 'NU')),
  ((20, 'PS'), (20, 'YA')),
  ((17, 'FX'), (17, 'EJ')),
  ((14, 'JM'), (13, 'XK')),
  ((11, 'KZ'), (3, 'RG')),
  ((8, 'UX'), (0, 'DG')),
  ((5, 'DO'), None),
  ((2, 'PH'), None),
  ((2, 'remain here'), None)])

## Day 17

In [85]:
day17Data = getData(17).strip()
rocks = [["@@@@"], [" @ ", "@@@", " @ "], ["@@@", "  @", "  @"], ["@", "@", "@", "@"], ["@@", "@@"]]
rocks = [["  " + y + " "*(5-len(y)) for y in x] for x in rocks]

In [86]:
rocks

[['  @@@@ '],
 ['   @   ', '  @@@  ', '   @   '],
 ['  @@@  ', '    @  ', '    @  '],
 ['  @    ', '  @    ', '  @    ', '  @    '],
 ['  @@   ', '  @@   ']]

In [87]:
cave = ["-------"]
streamI = 0
for rockNum in range(2022): #2022
    r = rocks[rockNum % 5]
    highestRock = next(i for i, row in enumerate(cave) if any(y != ' ' for y in row))
    for _ in range(len(r) + 3 - highestRock): cave.insert(0, "       ")
    for rockI, rockRow in enumerate(r):
        cave[len(r) - rockI - 1 + max(0, highestRock - 3 - len(r))] = rockRow
    canMoveDown = True
    rockMaxLoc = len(r) + max(0, highestRock - 3 - len(r))
    #print("\n".join(cave))
    while canMoveDown:
        rockFound = lastRockFound = False
        i = 0
        # can blow rock?
        canBlowRock = True
        while (not rockFound or not lastRockFound) and canBlowRock:
            rockFound = rockFound or "@" in cave[i]
            lastRockFound = rockFound and not "@" in cave[i]
            if "@" in cave[i]:
                ci = cave[i] if day17Data[streamI % len(day17Data)] == '<' else cave[i][::-1]
                canBlowRock &= ci.index("@") - 1 >= 0 and ci[ci.index("@") - 1] == ' '
            i += 1
        if canBlowRock:
            for i in range(rockMaxLoc - len(r), rockMaxLoc):
                if "@" in cave[i]:
                    ci = cave[i] if day17Data[streamI % len(day17Data)] == '<' else cave[i][::-1]
                    ci = ci[:max(0, ci.index("@") - 1)] + "".join(x for x in ci if x == '@') + " " + \
                                ci[ci.index("@") + sum(1 for x in ci if x == '@'):]
                    cave[i] = ci if day17Data[streamI % len(day17Data)] == '<' else ci[::-1]
        streamI += 1
        # move down
        canMoveDown = all(cave[i+1][k] in (" ", "@") for i in range(rockMaxLoc - len(r), rockMaxLoc)
                          for k, b in enumerate(cave[i]) if b == '@')
        if canMoveDown:
            for i in range(rockMaxLoc, rockMaxLoc - len(r), -1):
                if "@" in cave[i-1]:
                    cave[i] = "".join(x if x != "@" else " " for x in cave[i][:cave[i-1].index("@")]) + \
                              "".join(x for x in cave[i-1] if x == '@') + \
                              "".join(x if x != "@" else " " for x in cave[i][cave[i-1].index("@") + 
                                                                              sum(1 for x in cave[i-1] if x == '@'):])
            cave[rockMaxLoc-len(r)] = "".join(x if x != "@" else " " for x in cave[rockMaxLoc-len(r)])
            rockMaxLoc += 1
        else:
            # change rock character
            for i in range(rockMaxLoc - len(r), rockMaxLoc):
                cave[i] = "".join(x if x != "@" else "#" for x in cave[i])

In [88]:
len(cave) - next(i for i, row in enumerate(cave) if any(y != ' ' for y in row)) - 1

3147

In [89]:
cave = ["-------"]
streamI = 0
fullRows = []
periodFound = -1
for rockNum in range(1, 20000): # arbitrary right limit, will stop much earlier
    r = rocks[(rockNum - 1) % 5]
    highestRock = next(i for i, row in enumerate(cave) if any(y != ' ' for y in row))
    for _ in range(len(r) + 3 - highestRock): cave.insert(0, "       ")
    for rockI, rockRow in enumerate(r):
        cave[len(r) - rockI - 1 + max(0, highestRock - 3 - len(r))] = rockRow
    canMoveDown = True
    rockMaxLoc = len(r) + max(0, highestRock - 3 - len(r))
    #print("\n".join(cave))
    while canMoveDown:
        rockFound = lastRockFound = False
        i = 0
        # can blow rock?
        canBlowRock = True
        while (not rockFound or not lastRockFound) and canBlowRock:
            rockFound = rockFound or "@" in cave[i]
            lastRockFound = rockFound and not "@" in cave[i]
            if "@" in cave[i]:
                ci = cave[i] if day17Data[streamI % len(day17Data)] == '<' else cave[i][::-1]
                canBlowRock &= ci.index("@") - 1 >= 0 and ci[ci.index("@") - 1] == ' '
            i += 1
        if canBlowRock:
            for i in range(rockMaxLoc - len(r), rockMaxLoc):
                if "@" in cave[i]:
                    ci = cave[i] if day17Data[streamI % len(day17Data)] == '<' else cave[i][::-1]
                    ci = ci[:max(0, ci.index("@") - 1)] + "".join(x for x in ci if x == '@') + " " + \
                                ci[ci.index("@") + sum(1 for x in ci if x == '@'):]
                    cave[i] = ci if day17Data[streamI % len(day17Data)] == '<' else ci[::-1]
        streamI += 1
        # move down
        canMoveDown = all(cave[i+1][k] in (" ", "@") for i in range(rockMaxLoc - len(r), rockMaxLoc)
                          for k, b in enumerate(cave[i]) if b == '@')
        if canMoveDown:
            for i in range(rockMaxLoc, rockMaxLoc - len(r), -1):
                if "@" in cave[i-1]:
                    cave[i] = "".join(x if x != "@" else " " for x in cave[i][:cave[i-1].index("@")]) + \
                              "".join(x for x in cave[i-1] if x == '@') + \
                              "".join(x if x != "@" else " " for x in cave[i][cave[i-1].index("@") + 
                                                                              sum(1 for x in cave[i-1] if x == '@'):])
            cave[rockMaxLoc-len(r)] = "".join(x if x != "@" else " " for x in cave[rockMaxLoc-len(r)])
            rockMaxLoc += 1
        else:
            # change rock character
            for i in range(rockMaxLoc - len(r), rockMaxLoc):
                cave[i] = "".join(x if x != "@" else "#" for x in cave[i])
    if periodFound < 0:
        try:
            fullRow = next(len(cave) - i for i in range(len(cave)) if cave[i] == '#######')
            if len(fullRows) == 0 or fullRow != fullRows[0][0]:
                fullRows.insert(0, (fullRow, rockNum))
                try:
                    repeats = next((i, rock) for i, rock in fullRows[1:] 
                                   if all(x == y for x, y in zip(cave[len(cave) - fullRow:1000], cave[len(cave) - i:])))
                    period = (rockNum - repeats[1], fullRow - repeats[0])
                    print("After %d rocks and %d rows, every %d rocks the cave repeats, with %d rows" %(
                          repeats[1], repeats[0], period[0], period[1]))
                    periodFound = ((1000000000000 - repeats[1]) % period[0])
                except StopIteration:
                    pass

        except StopIteration:
            pass
    elif periodFound > 1:
        periodFound -= 1
    else:
        height = ((1000000000000 - repeats[1]) // period[0]) *period[1] + repeats[0] + len(cave) - fullRow - \
                    next(i for i, row in enumerate(cave) if any(y != ' ' for y in row)) - 1
        print("Total height after 1000000000000 rocks: %d" %(height))
        break

After 1245 rocks and 1974 rows, every 1710 rocks the cave repeats, with 2620 rows
Total height after 1000000000000 rocks: 1532163742758


## Day 18

In [90]:
day18Data = [tuple(int(x) for x in y.split(",")) for y in getData(18).strip().split("\n")]

In [91]:
# ~1 second
sum(1 for x, y, z in day18Data 
    for xi, yi, zi in [(x-1, y, z), (x+1, y, z), (x, y-1, z), (x, y+1, z), (x, y, z-1), (x, y, z+1)]
    if (xi, yi, zi) not in day18Data)

3610

In [92]:
# Hmm, ~5 seconds
inside = [(xi, yi, zi) for xi in range(min(x for x, y, z in day18Data), max(x for x, y, z in day18Data) + 1)
         for yi in range(min(y for x, y, z in day18Data if x == xi), max(y for x, y, z in day18Data if x == xi) + 1)
         for zi in range(min(z for x, y, z in day18Data if x == xi and y == yi), 
                         max(z for x, y, z in day18Data if x == xi and y == yi) + 1)]
prevLen = -1
while len(inside) != prevLen:
    prevLen = len(inside)
    inside = [(x, y, z) for x, y, z in inside if (x, y, z) in day18Data or all((xi, yi, zi) in inside for 
               xi, yi, zi in [(x-1, y, z), (x+1, y, z), (x, y-1, z), (x, y+1, z), (x, y, z-1), (x, y, z+1)])]
    
sum(1 for x, y, z in day18Data 
    for xi, yi, zi in [(x-1, y, z), (x+1, y, z), (x, y-1, z), (x, y+1, z), (x, y, z-1), (x, y, z+1)]
    if (xi, yi, zi) not in inside)

2082

## Day 19

In [93]:
day19Data = [{what: {y.split(" ")[1] : int(y.split(" ")[0]) for y in x.split("costs ")[4-i].split(".")[0].split(" and ")}
              for i, what in enumerate(['geodeR', 'obsidianR', 'clayR', 'oreR'])} for x in getData(19).strip().split("\n")]

In [94]:
def processDay(blueprint, daysLeft, resources, bestCache, maxNeed):
    #print("daysLeft %d, resources: %s" %(daysLeft, resources))
    if daysLeft == 1:
        return resources['geode'] + resources['geodeR']
    if bestCache[daysLeft] >= resources['geode'] + int(daysLeft*(resources['geodeR'] + (daysLeft - 1)/2)):
        return 0
    buildingOptions = []
    for w, needed in blueprint.items():
        if (w not in maxNeed or resources[w] < maxNeed[w]) and all(resources[x] >= j for x, j in needed.items()):
            newResources = dict((x, i - needed[x] if x in needed else i) for x, i in resources.items())
            for ore in ['ore', 'clay', 'obsidian', 'geode']:
                newResources[ore] += resources[ore + 'R']
            newResources[w] += 1
            buildingOptions.append(newResources)
    buildingOptions.append(dict((x, i + resources[x + 'R'] if x + 'R' in resources else i) for x, i in resources.items()))
    m = max(processDay(blueprint, daysLeft - 1, res, bestCache, maxNeed) for res in buildingOptions)
    if bestCache[daysLeft] < m:
        bestCache[daysLeft] = m
    return m

In [95]:
### Unfortunately this takes a lot of time (~2h), so we won't run it every time.
if False: 
    resources = dict((y, 0) for x in ['ore', 'clay', 'obsidian', 'geode'] for y in (x, x + 'R'))
    resources['oreR'] = 1
    geodes = []
    for i, blueprint in enumerate(day19Data):
        maxNeed = {w + 'R': max(x[w] for x in blueprint.values() if w in x) for w in ['ore', 'clay', 'obsidian']}
        start = time.time()
        geodes.append(processDay(blueprint, 24, resources, {j: 0 for j in range(25)}, maxNeed))
        print("bpn: %d, geodes: %d, time: %.2f" %(i, geodes[-1], time.time() - start))
    sum((i+1)*g for i, g in enumerate(geodes))

In [96]:
res = """bpn: 0, geodes: 1, time: 104.47
bpn: 1, geodes: 7, time: 51.79
bpn: 2, geodes: 0, time: 70.44
bpn: 3, geodes: 0, time: 337.90
bpn: 4, geodes: 0, time: 424.16
bpn: 5, geodes: 0, time: 76.28
bpn: 6, geodes: 1, time: 50.70
bpn: 7, geodes: 0, time: 310.81
bpn: 8, geodes: 3, time: 16.12
bpn: 9, geodes: 2, time: 93.28
bpn: 10, geodes: 2, time: 719.25
bpn: 11, geodes: 1, time: 159.85
bpn: 12, geodes: 0, time: 684.94
bpn: 13, geodes: 6, time: 30.79
bpn: 14, geodes: 0, time: 327.87
bpn: 15, geodes: 0, time: 103.03
bpn: 16, geodes: 5, time: 134.45
bpn: 17, geodes: 2, time: 391.32
bpn: 18, geodes: 1, time: 476.45
bpn: 19, geodes: 0, time: 374.27
bpn: 20, geodes: 2, time: 119.81
bpn: 21, geodes: 1, time: 381.33
bpn: 22, geodes: 4, time: 27.48
bpn: 23, geodes: 3, time: 19.81
bpn: 24, geodes: 3, time: 50.29
bpn: 25, geodes: 2, time: 531.69
bpn: 26, geodes: 0, time: 360.57
bpn: 27, geodes: 9, time: 29.50
bpn: 28, geodes: 1, time: 382.54
bpn: 29, geodes: 2, time: 57.06
"""
res = {int(x.split(": ")[1].split(", ", 1)[0]) : {'geodes' : int(x.split("geodes: ")[1].split(", ")[0]),
                             'time' : float(x.split("time: ")[1])} for x in res.strip().split("\n")}

In [97]:
sum((i+1)*x['geodes'] for i, x in res.items())

1023

In [98]:
print("That took %.0fh%.0fm%.0fs" %((hrs := (mins := (sec := sum(x['time'] for x in res.values())) // 60) // 60), 
                                    (mins - (hrs*60)), (sec - mins*60)))

That took 1h54m58s


In [99]:
### Unfortunately this takes a lot of time (~2h), so we won't run it every time.
if False: 
    resources = dict((y, 0) for x in ['ore', 'clay', 'obsidian', 'geode'] for y in (x, x + 'R'))
    resources['oreR'] = 1
    geodes = []
    for i in range(3):
        maxNeed = {w + 'R': max(x[w] for x in day19Data[i].values() if w in x) for w in ['ore', 'clay', 'obsidian']}
        start = time.time()
        geodes.append(processDay(day19Data[i], 32, resources, {j: 0 for j in range(33)}, maxNeed))
        print("bpn: %d, geodes: %d, time: %.2f" %(i, geodes[-1], time.time() - start))
    geodes[0] * geodes[1] * geodes[2]

In [100]:
res = """bpn: 0, geodes: 26, time: 2185.42
bpn: 1, geodes: 52, time: 480.82
bpn: 2, geodes: 10, time: 5737.45"""
res = {int(x.split(": ")[1].split(", ", 1)[0]) : {'geodes' : int(x.split("geodes: ")[1].split(", ")[0]),
                             'time' : float(x.split("time: ")[1])} for x in res.strip().split("\n")}

In [101]:
res[0]['geodes']*res[1]['geodes']*res[2]['geodes']

13520

In [102]:
print("That took %.0fh%.0fm%.0fs" %((hrs := (mins := (sec := sum(x['time'] for x in res.values())) // 60) // 60), 
                                    (mins - (hrs*60)), (sec - mins*60)))

That took 2h20m4s


## Day 20

In [103]:
class Number(object):
    def __init__(self, num, origPos): self.num, self.pos = num, origPos
    def __repr__(self): return "<Number(%d, original position: %d)" %(self.num, self.pos)

In [104]:
day20data = [Number(int(x), i) for i, x in enumerate(getData(20).strip().split("\n"))]
for i in range(len(day20data)):
    j, num = next((j, x) for j, x in enumerate(day20data) if x.pos == i)
    day20data.pop(j)
    toInsert = (j + num.num) % len(day20data)
    day20data.insert(toInsert if toInsert > 0 else len(day20data), num)

In [105]:
firstzero = next(j for j, x in enumerate(day20data) if x.num == 0)
sum(day20data[(i*1000+firstzero) % len(day20data)].num for i in range(1, 4))

5962

In [106]:
day20data = [Number(int(x)*811589153, i) for i, x in enumerate(getData(20).strip().split("\n"))]
for _ in range(10):
    for i in range(len(day20data)):
        j, num = next((j, x) for j, x in enumerate(day20data) if x.pos == i)
        day20data.pop(j)
        toInsert = (j + num.num) % len(day20data)
        day20data.insert(toInsert if toInsert > 0 else len(day20data), num)

In [107]:
firstzero = next(j for j, x in enumerate(day20data) if x.num == 0)
sum(day20data[(i*1000+firstzero) % len(day20data)].num for i in range(1, 4))

9862431387256

## Day 21

In [108]:
day21data = {x.split(": ")[0] : x.split(": ", 1)[1].replace("/", "//") for x in getData(21).strip().split("\n")}

In [109]:
def solveEquations(solved, notsolved):
    for k, v in solved.items():
        for k2 in notsolved:
            notsolved[k2] = notsolved[k2].replace(k, v)
    prevLen = -1
    while len(notsolved) != prevLen:
        prevLen = len(notsolved)
        for k in list(notsolved):
            try:
                sol = str(eval(notsolved[k]))
                solved[k] = sol
                del notsolved[k]
                for k2 in notsolved:
                    notsolved[k2] = notsolved[k2].replace(k, sol)
            except NameError:
                pass

In [110]:
solved = {k : v for k, v in day21data.items() if re.match("-?\d+", v)}
notsolved = {k : v for k, v in day21data.items() if k not in solved}
solveEquations(solved, notsolved)

In [111]:
solved['root']

'169525884255464'

In [112]:
def flipeqn(k, v):
    funrep = {'*' : '//', '//' : '*', '+' : '-', '-' : '+'}
    m = re.match("([a-z]+) ([\*/\+-]+) (-?\d+)", v)
    if m:
        otherk, fun, num = m.groups()
        return (otherk, k + " " + funrep[fun] + " " + num)
    else:
        m = re.match("(-?\d+) ([\*/\+-]+) ([a-z]+)", v)
        num, fun, otherk = m.groups()
        return (otherk, ((num + " " + fun + " " + k) if fun in ('//', '-') else (k + " " + funrep[fun] + " " + num)))
    raise ValueError

In [113]:
solved = {k : v for k, v in day21data.items() if re.match("-?\d+", v)}
notsolved = {k : v for k, v in day21data.items() if k not in solved}
del solved['humn']
solveEquations(solved, notsolved)

m = re.match("([a-z]+) . (-?\d+)", notsolved.pop('root'))
notsolved = dict(flipeqn(k, v) for k, v in notsolved.items())
solved[m.group(1)] = m.group(2)

for k2 in notsolved:
    notsolved[k2] = notsolved[k2].replace(m.group(1), m.group(2))
solveEquations(solved, notsolved)

In [114]:
solved['humn']

'3247317268284'

## Day 22

In [115]:
day22map, day22directions = getData(22).rstrip().split("\n\n")
day22map = day22map.split("\n") # day22map[y][x]
day22map = [x + " "*(max(len(y) for y in day22map) - len(x)) for x in day22map]
day22directions = [x for y in zip(re.split("\d+", day22directions), re.split("[RL]", day22directions)) for x in y][1:]

In [116]:
cy, cx = (0, next(i for i, x in enumerate(day22map[0]) if x == '.'))
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)] #y, x
dirstr = ">v<^."
cd, dy, dx = 0, 0, 1
for ii in range(len(day22directions)):
    if ii % 2 == 0:
        ins = int(day22directions[ii])
        di = 0
        while di < ins:
            day22map[cy] = day22map[cy][:cx] + dirstr[cd] + day22map[cy][cx+1:]
            ny, nx = (cy + dy) % len(day22map), (cx + dx) % len(day22map[(cy + dy) % len(day22map)])
            nextTile = day22map[ny][nx]
            if nextTile in dirstr:
                di += 1
                cy, cx = ny, nx
            elif nextTile == '#':
                break
            elif nextTile == ' ':
                ny, nx = next((nny, nnx) for i in range(10000)
                    if day22map[nny := ((ny + i*dy) %len(day22map))][nnx := ((nx + i*dx) % len(day22map[nny]))] != ' ')
                if day22map[ny][nx] in dirstr:
                    di += 1
                    cy, cx = ny, nx
                elif day22map[ny][nx] == '#':
                    break
                else:
                    raise ValueError("should never get here")
    else:
        cd = (cd + (1 if day22directions[ii] == 'R' else -1)) %4
        dy, dx = directions[cd]

In [117]:
1000*(cy+1) + 4*(cx+1) + cd

88226

In [118]:
#display(HTML("<style>pre { white-space: pre !important; }</style>"))
#print("\n".join("%3d %s" %x for x in enumerate(day22map)))

In [119]:
##general solution
## transform any dice into 
## left top   right
## .    front
## .    bott
## .    back

## then apply a general solution to this setup.

facelen = 50
dice = [" "*(facelen*3) for y in range(facelen)] + [" "*(facelen*2) for y in range(facelen*3)]
# map existing layout to this face. This should be possible, since we can take the top left item to be in the top frame,
# which then means there is a deterministic setup of where the others should be. Still messy though!


In [120]:
day22map, day22directions = getData(22).rstrip().split("\n\n")
day22map = day22map.split("\n") # day22map[y][x]
day22map = [x + " "*(max(len(y) for y in day22map) - len(x)) for x in day22map]
day22directions = [x for y in zip(re.split("\d+", day22directions), re.split("[RL]", day22directions)) for x in y][1:]

In [121]:
## by inspection, my layout is 
## .    top  right
## .    fron .
## left bott .
## back .    .
cy, cx, cs = (0, next(i for i, x in enumerate(day22map[0]) if x == '.'), 50)
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)] #y, x
dirstr = ">v<^."
cd, dy, dx = 0, 0, 1
for ii in range(len(day22directions)):
    if ii % 2 == 0:
        ins = int(day22directions[ii])
        di = 0
        while di < ins:
            day22map[cy] = day22map[cy][:cx] + dirstr[cd] + day22map[cy][cx+1:]
            nd = cd
            ## this is specific to my folding of the cube, every input is folded differently I would guess
            if cy in range(0, 50):
                if cx in range(50, 100):
                    if cy + dy in range(0, 50):
                        if cx + dx in range(50, 100): # inside Top
                            ny, nx = cy + dy, cx + dx
                        elif cx + dx == 49: #walk of the left of Top, arrive Left facing right
                            ny, nx, nd = 149 - cy, 0, 0
                        elif cx + dx == 100: #walk of the right of Top, arrive Right facing right
                            ny, nx = cy + dy, cx + dx
                    elif cy + dy == -1: #walk of the top of Top, arrive back facing right
                        ny, nx, nd = 100 + cx, 0, 0
                    elif cy + dy == 50: #walk of the bottom of Top, arrive Front from top
                        ny, nx = 50, cx
                elif cx in range(100, 150):
                    if cy + dy in range(0, 50):
                        if cx + dx in range(100, 150):
                            ny, nx = cy + dy, cx + dx # inside Right
                        elif cx + dx == 99: # walk of the left of Right, arrive Top facing left
                            ny, nx = cy, 99
                        elif cx + dx == 150: # walk of the right of Right, arrive Bottom facing left
                            ny, nx, nd = 149 - cy, 99, 2
                    elif cy + dy == -1: #walk of the top of Right, arrive Back facing upwards
                        ny, nx = 199, cx - 100
                    elif cy + dy == 50: #walk of the bottom of Right, arrive Front facing left
                        ny, nx, nd = cx - 50, 99, 2
            if cy in range(50, 100): 
                if cx in range(50, 100):
                    if cy + dy in range(50, 100):
                        if cx + dx in range(50, 100): # inside Front
                            ny, nx = cy + dy, cx + dx
                        elif cx + dx == 49: # walk of the left of Front, arrive Left facing down
                            ny, nx, nd = 100, cy - 50, 1
                        elif cx + dx == 100: # walk of the right of Front, arrive Right facing up
                            ny, nx, nd = 49, cy + 50, 3
                    elif cy + dy == 49: # walk of the top of Front, arrive Top facing up
                        ny, nx = 49, cx
                    elif cy + dy == 100: # walk of the bottom of Front, arrive Bottom facing down
                        ny, nx = 100, cx
            if cy in range(100, 150): ## double checked till here
                if cx in range(0, 50):
                    if cy + dy in range(100, 150):
                        if cx + dx in range(0, 50): # inside Left
                            ny, nx = cy + dy, cx + dx
                        elif cx + dx == -1: # walk of the left of Left, arrive Top facing right
                            ny, nx, nd = 149 - cy, 50, 0
                        elif cx + dx == 50: # walk of the right of Left, arrive Bottom facing right
                            ny, nx = cy, 50
                    elif cy + dy == 99: # walk of the top of left, arrive Front facing right
                        ny, nx, nd = cx + 50, 50, 0
                    elif cy + dy == 150: # walk of the bottom of Left, arrive Back facing down
                        ny, nx = 150, cx
                elif cx in range(50, 100):  ## double checked till here
                    if cy + dy in range(100, 150):
                        if cx + dx in range(50, 100): # inside Bottom
                            ny, nx = cy + dy, cx + dx
                        elif cx + dx == 49: # walk of the left of Bottom, arrive Left facing left
                            ny, nx = cy, 49
                        elif cx + dx == 100: # walk of the right of Bottom, arrive Right facing left
                            ny, nx, nd = 149 - cy, 149, 2
                    elif cy + dy == 99: #walk of the top of Bottom, arrive Front facing upwards
                        ny, nx = 99, cx
                    elif cy + dy == 150: #walk of the bottom of Bottom , arrive Back facing left
                        ny, nx, nd = cx + 100, 49, 2
            if cy in range(150, 200):
                if cx in range(0, 50):
                    if cy + dy in range(150, 200):
                        if cx + dx in range(0, 50): # inside Back
                            ny, nx = cy + dy, cx + dx
                        elif cx + dx == -1: # walk of the left of Back, arrive Top facing down
                            ny, nx, nd = 0, cy - 100,  1
                        elif cx + dx == 50: # walk of the right of Back, arrive Bottom facing up
                            ny, nx, nd = 149, cy - 100, 3
                    elif cy + dy == 149: # walk of the top of Back, arrive Left facing up
                        ny, nx = 149, cx
                    elif cy + dy == 200: # walk of the bottom Back, arrive Right facing down
                        ny, nx = 0, cx + 100
            nextTile = day22map[ny][nx]
            if nextTile in dirstr:
                di += 1
                cy, cx, dy, dx, cd = ny, nx, directions[nd][0], directions[nd][1], nd
            elif nextTile == '#':
                break
            elif nextTile == ' ':
                raise ValueError("should never get here")
    else:
        cd = (cd + (1 if day22directions[ii] == 'R' else -1)) %4
        dy, dx = directions[cd]
    #if ii == 20:
    #    break
1000*(cy+1) + 4*(cx+1) + cd

57305

## Day 23

In [122]:
day23data = [list(x) for x in getData(23).strip().split("\n")]
dirs = [((-1, -1), (-1, 0), (-1, 1)), ((1, -1), (1, 0), (1, 1)), ((-1, -1), (0, -1), (1, -1)), ((-1, 1), (0, 1), (1, 1))]
for i in range(10):
    #expands data with empty row
    if any(x != "." for x in day23data[0]): day23data.insert(0, ["."]*len(day23data[0]))
    if any(x != "." for x in day23data[-1]): day23data.append(["."]*len(day23data[0]))
    if any(x[0] != "." for x in day23data): day23data = [["."] + x for x in day23data]
    if any(x[-1] != "." for x in day23data): day23data = [x + ["."] for x in day23data]
    # propose moves
    proposedMoves = {}
    for yi, xi in itertools.product(range(1,len(day23data) - 1), range(1, len(day23data[0]) - 1)):
        if day23data[yi][xi] == '#':
            if any(day23data[yi + dy][xi + dx] == '#' for z in dirs for dy, dx in z):
                try:
                    pm = next(z[1] for z in dirs if all(day23data[yi + dy][xi + dx] == '.' for dy, dx in z))
                    proposedMoves[(yi, xi)] = (yi + pm[0], xi + pm[1])
                except StopIteration:
                    pass
    # prune moves
    validDestinations = [x for x, y in collections.Counter(proposedMoves.values()).items() if y == 1]
    proposedMoves = dict(((f, t) for f, t in proposedMoves.items() if t in validDestinations))
    # move
    for (fy, fx), (ty, tx) in proposedMoves.items():
        day23data[fy][fx] = "."
        day23data[ty][tx] = "#"
    dirs.append(dirs.pop(0))

In [123]:
print("\n".join(("".join(x) for x in day23data)))

....................................................................................
..........................#..#.........#..................#.#.........#.............
..........#...##...............#....#....#.#....#.......#........#.......#...#......
.......#.........#.##.##..#........#..#............#.#............#......#..........
.........#..#.#........#.#.#.##.....#.....#..#..#.......##..###..#.....##...#...#...
..............#...#.#............##..#...#....#...#...#...##.....#..#..#..#.....#...
.....#.#.#..#...#.....#.##.#####...#...#...##.#......#..#...##..##.....#..##..#.....
...........#....#..#.#.#..#......#...#..#.#.....##.#...#.#.#..##....##...#..........
....#..#..#.####.#....#.##.#.#..#.#.#.##.#.#.#.#...#.#..#.#.#...#.#...#.#.#...##.#..
.....#...#.#..#.#.#.#..#..#.#.#..#.#.#..#.#..##.###.#.##.#.#.###.#.###...#.#.#......
..........##.#...#.#..#..#.#.#.##.#.#.##.#..#..##.#.###...#.#.###.....#.#.#.#..#....
.......#.##.#..#..####.##.#..#.#.#.#.#.#...#.##.##.#.#...#.#.##.#

In [124]:
ymin = next(yi for yi, row in enumerate(day23data) if any(x == '#' for x in row))
ymax = next(len(day23data) - yi - 1 for yi, row in enumerate(day23data[::-1]) if any(x == '#' for x in row))
xmin = next(xi for xi in range(len(day23data[0])) if any(x[xi] == '#' for x in day23data))
xmax = next(xi for xi in range(len(day23data[0])-1, -1, -1) if any(x[xi] == '#' for x in day23data))

In [125]:
sum(1 for yi, xi in itertools.product(range(ymin, ymax + 1), range(xmin, xmax + 1)) if day23data[yi][xi] == '.')

4000

In [126]:
day23data = [list(x) for x in getData(23).strip().split("\n")]
dirs = [((-1, -1), (-1, 0), (-1, 1)), ((1, -1), (1, 0), (1, 1)), ((-1, -1), (0, -1), (1, -1)), ((-1, 1), (0, 1), (1, 1))]
ri = 1
while True:
    #expands data with empty row
    if any(x != "." for x in day23data[0]): day23data.insert(0, ["."]*len(day23data[0]))
    if any(x != "." for x in day23data[-1]): day23data.append(["."]*len(day23data[0]))
    if any(x[0] != "." for x in day23data): day23data = [["."] + x for x in day23data]
    if any(x[-1] != "." for x in day23data): day23data = [x + ["."] for x in day23data]
    # propose moves
    proposedMoves = {}
    for yi, xi in itertools.product(range(1,len(day23data) - 1), range(1, len(day23data[0]) - 1)):
        if day23data[yi][xi] == '#':
            if any(day23data[yi + dy][xi + dx] == '#' for z in dirs for dy, dx in z):
                try:
                    pm = next(z[1] for z in dirs if all(day23data[yi + dy][xi + dx] == '.' for dy, dx in z))
                    proposedMoves[(yi, xi)] = (yi + pm[0], xi + pm[1])
                except StopIteration:
                    pass
    # prune moves
    validDestinations = [x for x, y in collections.Counter(proposedMoves.values()).items() if y == 1]
    proposedMoves = dict(((f, t) for f, t in proposedMoves.items() if t in validDestinations))
    if len(proposedMoves) == 0:
        break
    # move
    for (fy, fx), (ty, tx) in proposedMoves.items():
        day23data[fy][fx] = "."
        day23data[ty][tx] = "#"
    dirs.append(dirs.pop(0))
    ri += 1

In [127]:
ri

1040

In [128]:
display(HTML("<style>pre { white-space: pre !important; }</style>"))
print("\n".join(("".join(x) for x in day23data)))

...............................................................................................................................................
......................................................#........................................................................................
...............................................#...............................................................................................
..........................................................#.#..................................................................................
............................#.........#.#.#.#...#...#.#.#..........#...........................................................................
...............................#.#........................#.#.#.#......#.#.....#...............................................................
............................#......#.#....#....#..#.....#............#.....#.#..........................................................

## Day 24

In [129]:
day24data = getData(24).strip().split("\n")

In [130]:
## Just to make sure that no blizzards leave through entry and exit. If so, this solution doesn't work
assert not any(c in ['^', 'v'] for r in day24data for i in (1, -2) for c in r[i])

In [131]:
def analyseBlizzards(m):
    Y, X = len(m) - 2, len(m[0]) - 2
    blizz = [[(yi, xi) for yi, row in enumerate(m[1:-1]) for xi, x in enumerate(row[1:-1]) if x == w] 
             for w in ['<', '>', '^', 'v']]
    cacheState = dict()
    def validMoves(i, y, x):
        if i in cacheState: # could really calculate LCM(35, 100) = 700 - every 700 iterations does the state repeat.
            ms = cacheState[i]
        else:
            ms = [(((yi + i*dy) % Y), (xi + i*dx) % X) for bs, (dy, dx) 
                in zip(blizz, [(0, -1), (0, 1), (-1, 0), (1, 0)]) for yi, xi in bs]
            ms = [[True] + [False]*(X-1)] + [[(yi, xi) not in ms for xi in range(X)] for yi in range(Y)] + \
                    [[False]*(X-1) + [True]]
            cacheState[i] = ms
        vm = [(y + dy, x + dx) for dy, dx in [(0, -1), (0, 1), (-1, 0), (1, 0), (0, 0)] if x + dx > -1 and x + dx < X and
              y + dy < Y + 2 and ms[y + dy][x + dx]]
        return vm
    return validMoves

In [132]:
def moveThroughBlizzard(start, dest, i=0):
    validMoves = analyseBlizzards(day24data)
    queue = [(i, start)]
    while queue:
        i, (y, x) = queue.pop()
        vms = validMoves(i, y, x)
        if dest in vms:
            return i
        else:
            for ny, nx in vms:
                if (nq := (i+1, (ny, nx))) not in queue: #the sorting order is key here!
                    bisect.insort_left(queue, nq, key=lambda a: abs(a[1][0] - start[0]) + abs(a[1][1] - start[1]) - a[0])

In [133]:
sol = moveThroughBlizzard((0, 0), (len(day24data) - 1, len(day24data[0]) - 3))
sol

251

In [134]:
sol2 = moveThroughBlizzard((len(day24data) - 1, len(day24data[0]) - 3), (0, 0), sol)
sol3 = moveThroughBlizzard((0, 0), (len(day24data) - 1, len(day24data[0]) - 3), sol2)
sol3

758

## Day 25

In [135]:
day25data = getData(25).strip().split("\n")

In [136]:
def toDecimal(s):
    cmap = {'2' : 2, '1' : 1, '0' : 0, '-' : -1, '=' : -2}
    return sum(5**i*cmap[c] for i, c in enumerate(s[::-1]))

In [137]:
def toSNAFU(n): # only for positive n
    sn = ""
    while n > 0:
        n, s = divmod(n, 5)
        if s == 3 or s == 4:
            n += 1
            s = "-" if s == 4 else "="
        sn += str(s)
    return sn[::-1]

In [138]:
toSNAFU(sum(toDecimal(x) for x in day25data))

'2=0-2-1-0=20-01-2-20'